# 使い方・このノートブックについて

- Google Colaboratoryで使用することを想定しています。
- GPUインスタンスで実行してください
- 実行前にdatasetフォルダ内に、データセットへのリンクとしてtrainとtestという名前でシンボリックリンクを作成してください



# 環境構築

## 入力ファイル・出力ファイルのディレクトリを指定するファイルを作成
__セル内を適宜かきかえること__

In [0]:
with open('.env',"w") as f:
    f.write("dataRootDir = path/to/inputData\r\n")
    f.write("outputDir = ./output")

import os
d = []
for  line in open('.env'):
    if not line.__contains__('=') : continue
    d.append(line.strip().split(' = '))
d = dict(d)
os.environ['dataRootDir'] = d['dataRootDir']
dataRootDir = d['dataRootDir']
os.environ['outputDir'] = d['outputDir']
outputDir = d['outputDir']
print(d)

In [0]:
!mkdir /content/VoicePix2Pix/dataset


In [0]:
#@title Example form fields
#@markdown ## GoogleDrivedatasetのあるフォルダ
#@markdown Googleドライブをマウントしてから左ペインのファイルタブから探すとよい
dataset_train = 'My Drive/dataset/audio/yuduki-yukari/wagahai_ha_neko'  #@param {type: "string"}
dataset_test = 'My Drive/dataset/audio/hiroshiba_normal_48kHz'  #@param {type: "string"}
output = 'My Drive/Colab Notebooks/VoicePix2Pix'  #@param {type: "string"}
#@markdown ---


import os
dataset_train = os.path.join('/content/drive/',dataset_train) + "/"
dataset_test = os.path.join('/content/drive/',dataset_test) + "/"
output = os.path.join('/content/drive/',output) + "/"

os.environ['path_train'] = dataset_train
os.environ['path_test'] = dataset_test
os.environ['path_output'] = output



In [2]:
# Googleドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

# Chainer/CuPy Installer for Google Colaboratory
# https://github.com/kmaehashi/chainer-colab
!curl https://colab.chainer.org/install | sh -

#world4py インストール
%cd /content
!git clone --depth 1 https://github.com/yamachu/WORLD4py
%cd /content/WORLD4py
!python setup.py install

# VoicePix2Pixをclone
%cd /content
!git clone --depth 1 https://github.com/lilacs2039/VoicePix2Pix
%cd /content/VoicePix2Pix
!pip install -r requirements.txt

# シンボリックリンクを作成
!rm /content/VoicePix2Pix/dataset/train
!ln -s "$path_train" /content/VoicePix2Pix/dataset/train
!rm /content/VoicePix2Pix/dataset/test
!ln -s "$path_test" /content/VoicePix2Pix/dataset/test
!rm /content/VoicePix2Pix/output
!ln -s "$path_output" /content/VoicePix2Pix/output

%cd /content/VoicePix2Pix


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1379  100  1379    0     0  10446      0 --:--:-- --:--:-- --:--:-- 10446
+ apt -y -q install cuda-libraries-dev-9-2
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  cuda-cublas-dev-9-2 cuda-cufft-dev-9-2 cuda-curand-dev-9-2
  cuda-cusolver-dev-9-2 cuda-cusparse-dev-9-2 cuda-npp-dev-9-2
  cuda-nvgraph-dev-9-2 cuda-nvrtc-dev-9-2
The following NEW packages will be installed:
  cuda-cublas-dev-9-2 cuda-cufft-dev-9-2 cuda-curand-dev-9-2
  cuda-cusolver-dev-9-2 cuda-cusparse-dev-9-2 cuda-libraries-dev-9-2
  cuda-npp-dev-9-2 cuda-nvgraph-dev-9-2 cuda-nvrtc-dev-9-2
0 upgraded, 9 newly installed, 0 to remove and 3 not up

# 学習開始

In [22]:
#GPUありで学習
%cd /content/VoicePix2Pix
!python train.py -g 0 -i ./dataset --out ./output --snapshot_interval 1600


/content/VoicePix2Pix
GPU: 0
# Minibatch-size: 1
# epoch: 200

search dataset paths
    from: ./dataset/train
load dataset paths done
search dataset paths
    from: ./dataset/test
load dataset paths done
/usr/local/lib/python3.6/dist-packages/chainer/iterators/multiprocess_iterator.py:31: TimeoutWarning: Stalled dataset is detected. See the documentation of MultiprocessIterator for common causes and workarounds:
https://docs.chainer.org/en/stable/reference/generated/chainer.iterators.MultiprocessIterator.html
  MultiprocessIterator.TimeoutWarning)
/content/VoicePix2Pix/util.py:71: RuntimeWarning: divide by zero encountered in log10
  Dabs = np.log10(np.abs(D))
/content/VoicePix2Pix/util.py:119: RuntimeWarning: invalid value encountered in multiply
  scaled = ndArray * scale_factor
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:29: RuntimeWarning: invalid value encountered in reduce
  return umr_minimum(a, axis, None, out, keepdims)
Exception in main training loop: out of

In [21]:
!pip install librosa



    100% |████████████████████████████████| 1.6MB 9.9MB/s 
    100% |████████████████████████████████| 327kB 14.1MB/s 
    100% |████████████████████████████████| 3.2MB 11.2MB/s 
    100% |████████████████████████████████| 16.1MB 2.7MB/s 
  Running setup.py bdist_wheel for librosa ... - \ done
  Stored in directory: /root/.cache/pip/wheels/18/b8/10/f0f8f6ac60668a5cd75596cf14c25bb6b3ea1ecd815f058b7e
  Running setup.py bdist_wheel for audioread ... - done
  Stored in directory: /root/.cache/pip/wheels/53/02/90/7b5c4081b7470c550ab605f600bad237dde12a6b8999b11f50
  Running setup.py bdist_wheel for resampy ... - \ done
  Stored in directory: /root/.cache/pip/wheels/ff/4f/ed/2e6c676c23efe5394bb40ade50662e90eb46e29b48324c5f9b
Successfully built librosa audioread resampy


In [0]:
print(os.environ['dataRootDir'])
!echo $dataRootDir

In [0]:
#GPUありで学習
cmd  = "python train_facade.py -g 0 -i {} --out {} --snapshot_interval 10000".format(d['dataRootDir'],d['outputDir'])
import subprocess
res = subprocess.call(cmd)
print(res)